In [25]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
spark=SparkSession.builder.appName('data_processing').getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df=spark.read.csv('s3://labsmolin19/COVID-19.csv',inferSchema=True,header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [48]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['fecha reporte web', 'ID de caso', 'Fecha de notificación', 'Código DIVIPOLA departamento', 'Nombre departamento', 'Código DIVIPOLA municipio', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Ubicación del caso', 'Estado', 'Código ISO del país', 'Nombre del país', 'Recuperado', 'Fecha de inicio de síntomas', 'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación', 'Pertenencia étnica', 'Nombre del grupo étnico']

In [49]:
#decido borrar todos los codigos ya que no me parecen importantes para hacer una consulta

df_new=df.drop('Fecha de notificación','ID de caso','fecha reporte web', 'Código DIVIPOLA departamento', 'Código DIVIPOLA municipio', 'Código ISO del país', 'Pertenencia étnica', 'Nombre del grupo étnico' ,'Ubicación del caso', 'Estado', 'Nombre del país','Fecha de inicio de síntomas', 'Fecha de muerte', 'Fecha de diagnóstico', 'Fecha de recuperación', 'Tipo de recuperación')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [50]:
df_new.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Nombre departamento', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Recuperado']

In [51]:
#agreago una nueva columna columna y las lleno con que fue examinado por smolin19
df_new_column = df_new.withColumn("Examinado", lit("Smolin19"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [52]:
df_new_column.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['Nombre departamento', 'Nombre municipio', 'Edad', 'Unidad de medida de edad', 'Sexo', 'Tipo de contagio', 'Recuperado', 'Examinado']

In [53]:
df_new_column.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+----+------------------------+----+----------------+----------+---------+
|Nombre departamento|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Recuperado|Examinado|
+-------------------+----------------+----+------------------------+----+----------------+----------+---------+
|             BOGOTA|          BOGOTA|  19|                       1|   F|       Importado|Recuperado| Smolin19|
|              VALLE|            BUGA|  34|                       1|   M|       Importado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  50|                       1|   F|       Importado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  55|                       1|   M|     Relacionado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  25|                       1|   M|     Relacionado|Recuperado| Smolin19|
|          ANTIOQUIA|          ITAGUI|  27|                       1|   F|     Relacionado|Recuperado| Sm

In [59]:
df_new_column.filter(df_new_column['Nombre municipio']=='MEDELLIN').show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+----------------+----+------------------------+----+----------------+----------+---------+
|Nombre departamento|Nombre municipio|Edad|Unidad de medida de edad|Sexo|Tipo de contagio|Recuperado|Examinado|
+-------------------+----------------+----+------------------------+----+----------------+----------+---------+
|          ANTIOQUIA|        MEDELLIN|  50|                       1|   F|       Importado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  55|                       1|   M|     Relacionado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  25|                       1|   M|     Relacionado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  26|                       1|   F|     Relacionado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  28|                       1|   M|     Relacionado|Recuperado| Smolin19|
|          ANTIOQUIA|        MEDELLIN|  55|                       1|   M|       Importado|Recuperado| Sm

In [60]:
df_new_column.groupBy('Recuperado').count().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+-------+
|Recuperado|  count|
+----------+-------+
| Fallecido|  36288|
|       N/A|   3016|
| fallecido|    296|
|Recuperado|1204452|
|    Activo|  64324|
+----------+-------+

In [63]:
write_uri='s3://labsmolin19/casos_medellin'

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [64]:
df_new_column.filter(df_new_column['Nombre municipio']=='MEDELLIN').write.format("csv").option("header","true").save(write_uri)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…